# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [31]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [32]:
filepath = ("../WeatherPy/Output File/city_data.csv")
city_data = pd.read_csv(filepath)
city_data.head()


,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,tres arroyos,-38.37,-60.28,58.48,45.0,0.0,14.07,AR,1.602704e+09
1,vaini,-21.20,-175.20,73.40,73.0,90.0,9.17,TO,1.602704e+09
2,rikitea,-23.12,-134.97,73.90,79.0,53.0,18.21,PF,1.602703e+09
3,esperance,-33.87,121.90,57.20,100.0,90.0,9.17,AU,1.602704e+09
4,puerto ayacucho,5.66,-67.62,89.64,60.0,0.0,4.72,VE,1.602704e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [33]:
cd_2 = city_data.dropna(how='any')

In [34]:
gmaps.configure(api_key=g_key)

In [35]:
locations = cd_2[['Latitude', 'Longitude']]
weights = cd_2['Humidity']
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [36]:
nice_weather = cd_2.drop(cd_2[(cd_2['Max Temp'] <= 75) | (cd_2['Max Temp'] >= 80) | (cd_2['Cloudiness'] >= 50) | (cd_2['Wind Speed'] >= 10)].index)

nice_weather.reset_index(drop=True)

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,hambantota,6.12,81.12,78.80,100.0,20.0,3.36,LK,1.602704e+09
1,kapaa,22.08,-159.32,80.60,74.0,1.0,3.38,US,1.602704e+09
2,kahului,20.89,-156.47,86.00,51.0,1.0,0.38,US,1.602704e+09
3,pacific grove,36.62,-121.92,86.00,56.0,1.0,5.82,US,1.602704e+09
4,arraial do cabo,-22.97,-42.03,78.80,74.0,47.0,8.05,BR,1.602704e+09
...,...,...,...,...,...,...,...,...,...
86,benghazi,32.12,20.07,72.09,58.0,0.0,6.80,LY,1.602705e+09
87,forestville,38.85,-76.88,75.00,37.0,1.0,4.70,US,1.602705e+09
88,vallenar,-28.57,-70.76,79.81,23.0,0.0,9.13,CL,1.602705e+09
89,baikunthpur,23.25,82.55,72.36,86.0,5.0,4.61,IN,1.602705e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [39]:

base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location='

for index, row in nice_weather.iterrows():
    city_name = row['City']
    city_lat = row['Latitude']
    city_long = row['Longitude']
    
    params = {
    "location": {city_lat,city_long},
    "radius" : "5000",
    "type" : "hotel",
    "key" : g_key,
    }
    
    
    hotel_df = requests.get(base_url, params=params).json()
    #time.sleep(1)
    print(json.dumps(hotel_df, indent=4))

hotel_df.head()


{
    "html_attributions": [],
    "results": [],
    "status": "INVALID_REQUEST"
}
{
    "html_attributions": [],
    "results": [],
    "status": "INVALID_REQUEST"
}
{
    "html_attributions": [],
    "results": [],
    "status": "INVALID_REQUEST"
}
{
    "html_attributions": [],
    "results": [],
    "status": "INVALID_REQUEST"
}
{
    "html_attributions": [],
    "results": [],
    "status": "INVALID_REQUEST"
}
{
    "html_attributions": [],
    "results": [],
    "status": "INVALID_REQUEST"
}
{
    "html_attributions": [],
    "results": [],
    "status": "INVALID_REQUEST"
}
{
    "html_attributions": [],
    "results": [],
    "status": "INVALID_REQUEST"
}
{
    "html_attributions": [],
    "results": [],
    "status": "INVALID_REQUEST"
}
{
    "html_attributions": [],
    "results": [],
    "status": "INVALID_REQUEST"
}
{
    "html_attributions": [],
    "results": [],
    "status": "INVALID_REQUEST"
}
{
    "html_attributions": [],
    "results": [],
    "status": "INVALID_REQ

KeyboardInterrupt: 

In [58]:
hotel_df = pd.DataFrame(nice_weather[["City", "Country", "Latitude", "Longitude"]].copy())
hotel_df["Hotel Name"] = ""
hotel_df


,City,Country,Latitude,Longitude,Hotel Name
9,hambantota,LK,6.12,81.12,
34,kapaa,US,22.08,-159.32,
45,kahului,US,20.89,-156.47,
50,pacific grove,US,36.62,-121.92,
57,arraial do cabo,BR,-22.97,-42.03,
...,...,...,...,...,...
575,benghazi,LY,32.12,20.07,
577,forestville,US,38.85,-76.88,
582,vallenar,CL,-28.57,-70.76,
590,baikunthpur,IN,23.25,82.55,


In [69]:
for index, row in nice_weather.iterrows():
    city_name = row['City']
    lat = row["Latitude"]
    lng = row["Longitude"]
    params = {
        "radius" : "5000",
        "type" : "hotel",
        "key" : g_key,
        "location" : f"{lat},{lng}"
      }
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
    hotels = requests.get(base_url, params=params)
   # convert to json

    hotels = hotels.json()
    #Grab the first hotel from the results and store the name
    try:        
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        print(f"Closest hotel to {city_name} is {hotels['results'][0]['name']}")
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")




Closest hotel to hambantota is Hambantota
Closest hotel to kapaa is Kapaʻa
Closest hotel to kahului is Kahului
Closest hotel to pacific grove is Monterey
Closest hotel to arraial do cabo is Cabo Frio


KeyboardInterrupt: 

In [64]:
hotel_df.head()

,City,Country,Latitude,Longitude,Hotel Name
9,hambantota,LK,6.12,81.12,Hambantota
34,kapaa,US,22.08,-159.32,Kapaʻa
45,kahului,US,20.89,-156.47,Kahului
50,pacific grove,US,36.62,-121.92,Monterey
57,arraial do cabo,BR,-22.97,-42.03,Cabo Frio


In [67]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [68]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))